### First Try and Initialization

In [2]:
from smolagents import CodeAgent, LiteLLMModel, DuckDuckGoSearchTool

In [4]:
model = LiteLLMModel(
    model_id="ollama_chat/qwen2.5:7b",  # Or try other Ollama-supported models
    api_base="http://127.0.0.1:11434",  # Default Ollama local server
    api_key="1122",
    num_ctx=8192,
)

In [5]:
agent = CodeAgent(tools=[DuckDuckGoSearchTool()], model=model, add_base_tools=True)


In [6]:
agent.run("What is the capital of India?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What is the capital of India?                                                                                   │
│                                                                                                                 │
╰─ LiteLLMModel - ollama_chat/qwen2.5:7b ─────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  result = web_search(query="capital of India")                                                                    
  print(result)                                                                                                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results

[New Delhi - Wikipedia](https://en.wikipedia.org/wiki/New_Delhi)
New Delhi (/ ˈ nj uː ˈ d ɛ. l i / ⓘ; [6] Hindi: Naī Dillī , pronounced [ˈnəiː ˈdɪlːiː]) is the capital of India and
a part of the National Capital Territory of Delhi (NCT). New Delhi is the seat of all three branches of the 
Government of India, hosting the Rashtrapati Bhavan, Sansad Bhavan, and the Supreme Court.New Delhi is a 
municipality within the NCT, administered by the New ...

[List of state and union territory capitals in India - 
Wikipedia](https://en.wikipedia.org/wiki/List_of_state_and_union_territory_capitals_in_India)
India is a federal constitutional republic governed under a parliamentary system consisting of 28 states and 8 
union territories. [1] All states, as well as the union territories of Jammu and Kashmir, Puducherry and the 
National Capital Territory of Delhi, have elected legislatures and governments, both patterned on the Westminster 
model.The remaining five union territories are directly ruled ...

[New Delhi | History, Population, Map, & Facts | Britannica](https://www.britannica.com/place/New-Delhi)
New Delhi, national capital of India. It is situated in the north-central part of the country on the west bank of 
the Yamuna River, adjacent to and just south of Delhi city (Old Delhi) and within the Delhi national capital 
territory. Learn more about New Delhi in this article.

[States and Capitals of India, 28 States and 8 UTs in India (Updated in 
...](https://www.careerpower.in/states-and-capitals-of-india.html)
States and Capitals of India. India is the 7th largest country in the world and also the 2nd most populous. India 
is a diverse and culturally rich Nation that comprises 28 states and 8 Union territories, each with its own unique 
identity, history, and administrative center. India lies in the southern Asia.

[Delhi | History, Population, Map, & Facts | Britannica](https://www.britannica.com/place/Delhi)
Delhi is a city and national capital territory in north-central India. The city of Delhi actually consists of two 
components: Old Delhi, in the north, the historic city; and New Delhi, in the south, since 1947 the capital of 
India, built in the first part of the 20th century as the capital of British India.

[List of capitals of India - Wikipedia](https://en.wikipedia.org/wiki/List_of_capitals_of_India)
In 1858, Allahabad (now Prayagraj) became the capital of India for a day when it also served as the capital of 
North-Western Provinces. [3]During the British Raj, until 1911, Calcutta was the capital of India. [4]By the latter
half of the 19th century, Shimla had become the summer capital. [5]King George V proclaimed the transfer of the 
capital from Calcutta to Delhi at the climax of the 1911 ...

[What is the Capital of India? - WorldAtlas](https://www.worldatlas.com/articles/what-is-the-capital-of-india.html)
Learn about the history, geography, and problems of New Delhi, the capital city of India. Find out why and how the 
capital was moved from Kolkata to New Delhi in 1911.

[What is the Capital of India? - Mappr](https://www.mappr.co/capital-cities/india/)
Learn about the history, geography, culture, and attractions of New Delhi, the capital of India. Find out how New 
Delhi became the capital in 1912, what is the difference between Delhi and New Delhi, and what is the climate and 
population of the city.

[What is the capital of India? States and union territories 
explained.](https://www.usatoday.com/story/news/world/2023/05/24/what-is-the-capital-of-india/70195720007/)
New Delhi is part of one of India's union territories, the National Capital Territory of Dehli. Old Delhi , now a 
historic city, is north of New Delhi in the same union territory.

[What is the capital city of India? - 
GeeksforGeeks](https://www.geeksforgeeks.org/what-is-the-capital-city-of-india/)
The capital city of India is New Delhi, situated in the northern part of the country. It was officially declared 
t

[Step 1: Duration 138.41 seconds| Input tokens: 2,141 | Output tokens: 40]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("New Delhi")                                                                                        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: New Delhi

[Step 2: Duration 76.10 seconds| Input tokens: 5,429 | Output tokens: 75]

'New Delhi'

## Messages to Prompt 

In [7]:
messages = [
    {"role": "system", "content": "You are an AI assistant with access to various tools."},
    {"role": "user", "content": "Hi !"},
    {"role": "assistant", "content": "Hi human, what can help you with ?"},
]

In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("HuggingFaceTB/SmolLM2-1.7B-Instruct")
rendered_prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

tokenizer_config.json:   0%|          | 0.00/3.76k [00:00<?, ?B/s]

c:\Users\saish\AI_Agents_Build\venv\lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\saish\.cache\huggingface\hub\models--HuggingFaceTB--SmolLM2-1.7B-Instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.json:   0%|          | 0.00/801k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

In [9]:
print(rendered_prompt)

<|im_start|>system
You are an AI assistant with access to various tools.<|im_end|>
<|im_start|>user
Hi !<|im_end|>
<|im_start|>assistant
Hi human, what can help you with ?<|im_end|>
<|im_start|>assistant



## Tool Creation

In [13]:
class Tool:
    """
    A class representing a reusable piece of code (Tool).

    Attributes:
        name (str): Name of the tool.
        description (str): A textual description of what the tool does.
        func (callable): The function this tool wraps.
        arguments (list): A list of argument.
        outputs (str or list): The return type(s) of the wrapped function.
    """
    def __init__(self,
                 name: str,
                 description: str,
                 func: callable,
                 arguments: list,
                 outputs: str):
        self.name = name
        self.description = description
        self.func = func
        self.arguments = arguments
        self.outputs = outputs

    def to_string(self) -> str:
        """
        Return a string representation of the tool,
        including its name, description, arguments, and outputs.
        """
        args_str = ", ".join([
            f"{arg_name}: {arg_type}" for arg_name, arg_type in self.arguments
        ])

        return (
            f"Tool Name: {self.name},"
            f" Description: {self.description},"
            f" Arguments: {args_str},"
            f" Outputs: {self.outputs}"
        )

    def __call__(self, *args, **kwargs):
        """
        Invoke the underlying function (callable) with provided arguments.
        """
        return self.func(*args, **kwargs)

In [20]:

def calculator(a: int, b: int) -> int:
    """Multiply two integers."""
    return a * b


In [15]:
Calculator_tool = Tool(
    "calculator",                   # name
    "Multiply two integers.",       # description
    calculator,                     # function to call
    [("a", "int"), ("b", "int")],   # inputs (names and types)
    "int",                          # output
)

In [22]:
print(Calculator_tool.to_string())

Tool Name: calculator, Description: Multiply two integers., Arguments: a: int, b: int, Outputs: int


## Weather API Call

In [25]:
from dotenv import load_dotenv
import os

load_dotenv()
Weather_API_KEY = os.getenv("WEATHER_API_KEY")

In [31]:
# Code Agent Example: Retrieve Weather Information
def get_weather(city):
    import requests
    api_url = f"https://api.weatherapi.com/v1/current.json?key={Weather_API_KEY}&q={city}&aqi=no"
    response = requests.get(api_url)
    if response.status_code == 200:
        data = response.json()
        return data.get("current", "No weather information available")
    else:
        return "Error: Unable to fetch weather data."

# Execute the function and prepare the final answer
result = get_weather("New York")
final_answer = f"The current weather in New York is: {result}"
print(final_answer)

The current weather in New York is: {'last_updated_epoch': 1743788700, 'last_updated': '2025-04-04 13:45', 'temp_c': 18.3, 'temp_f': 64.9, 'is_day': 1, 'condition': {'text': 'Sunny', 'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png', 'code': 1000}, 'wind_mph': 12.8, 'wind_kph': 20.5, 'wind_degree': 314, 'wind_dir': 'NW', 'pressure_mb': 1022.0, 'pressure_in': 30.18, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 36, 'cloud': 0, 'feelslike_c': 18.3, 'feelslike_f': 64.9, 'windchill_c': 14.6, 'windchill_f': 58.3, 'heatindex_c': 15.2, 'heatindex_f': 59.3, 'dewpoint_c': 9.3, 'dewpoint_f': 48.7, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 5.0, 'gust_mph': 16.2, 'gust_kph': 26.1}
